#### Calculate Distance Matrix

In [2]:
from collections import defaultdict
import pandas as pd
import requests
import json 

#solve module 'numpy' has no attribute 'ndarray'
import numpy as np
import math

In [3]:
def get_tasks(file: str, depot_coords: tuple, sheet: str):
    '''Processes a tasks Excel sheet into a JSON format for a POST request payload.
    Required format of the Bing REST API is a dictionary of origin and locations, where the value is another dictionary of the latitude and longitude.'''
    df = pd.read_excel(f"./{file}", sheet_name=sheet)
    #pointnum is the 10th column in excel sheet
    pointnum=int(df.columns[10])
    tasks_df = df.iloc[0:pointnum][:]
    n=len(tasks_df)
    locations = dict() 

    locations["origins"] = []
    locations["destinations"] = []
    locations["travelMode"] = "driving"


    # Adding depot location as location 0
    locations["origins"].append({"longitude": depot_coords[0], "latitude": depot_coords[1]})
    locations["destinations"].append({"longitude": depot_coords[0], "latitude": depot_coords[1]})

    for index, row in tasks_df.iterrows():
        if index in range(n):
            locations["origins"].append({"longitude": row['Longtitude'], "latitude": row['Latitude']})
            locations["destinations"].append({"longitude": row['Longtitude'], "latitude": row['Latitude']})
    return n, locations

In [4]:
def distance_matrix(response: requests.Response):
    '''Process the reponse to get the distance matrix.'''
    results = response.json()["resourceSets"][0]["resources"][0]["results"]
    n = int(math.sqrt(len(results)))
    
    matrix = [[np.Inf for i in range(n)] for j in range(n)]
    for entry in results:
        matrix[entry["originIndex"]][entry["destinationIndex"]] = entry["travelDistance"]
    
    return matrix

In [6]:
api_key = "Ag0unjZ8eW7hV91G8dXRk2Qf9Sk_GEIltJ9zLxdnwlP-s5wcZUQ1Q9x2yquA0eFf"
url = "https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key="+api_key
headers = {'Content-Type': 'application/json'} 


# Shepparton
n1, cluster1 = get_tasks("shepp_cluster.xlsx", (145.41880628382472, -36.37819524436343), "cluster 1")
response1 = requests.post(url, headers=headers, data=json.dumps(cluster1))

n2, cluster2 = get_tasks("shepp_cluster.xlsx", (145.41880628382472, -36.37819524436343), "cluster 2")
response2 = requests.post(url, headers=headers, data=json.dumps(cluster2))

n3, cluster3 = get_tasks("shepp_cluster.xlsx", (145.41880628382472, -36.37819524436343), "cluster 3")
response3 = requests.post(url, headers=headers, data=json.dumps(cluster3))

n4, cluster4 = get_tasks("shepp_cluster.xlsx", (145.41880628382472, -36.37819524436343), "cluster 4")
response4 = requests.post(url, headers=headers, data=json.dumps(cluster4))




In [6]:
col1 = []
for i in range(len(cluster1['origins'])):
     col1.append("({},{})".format(cluster1['origins'][i]['longitude'], cluster1['origins'][i]['latitude']))

col2 = []
for i in range(len(cluster2['origins'])):
     col2.append("({},{})".format(cluster2['origins'][i]['longitude'], cluster2['origins'][i]['latitude']))

col3 = []
for i in range(len(cluster3['origins'])):
     col3.append("({},{})".format(cluster3['origins'][i]['longitude'], cluster3['origins'][i]['latitude']))

col4 = []
for i in range(len(cluster4['origins'])):
     col4.append("({},{})".format(cluster4['origins'][i]['longitude'], cluster4['origins'][i]['latitude']))



In [7]:
dist_matrix1 = pd.DataFrame(distance_matrix(requests.post(url, headers=headers, data=json.dumps(cluster1))))
dist_matrix1.columns = col1
dist_matrix1 = dist_matrix1.set_index(pd.Index(col1))

dist_matrix2 = pd.DataFrame(distance_matrix(requests.post(url, headers=headers, data=json.dumps(cluster2))))
dist_matrix2.columns = col2
dist_matrix2 = dist_matrix2.set_index(pd.Index(col2))

dist_matrix3 = pd.DataFrame(distance_matrix(requests.post(url, headers=headers, data=json.dumps(cluster3))))
dist_matrix3.columns = col3
dist_matrix3 = dist_matrix3.set_index(pd.Index(col3))

dist_matrix4 = pd.DataFrame(distance_matrix(requests.post(url, headers=headers, data=json.dumps(cluster4))))
dist_matrix4.columns = col4
dist_matrix4 = dist_matrix4.set_index(pd.Index(col4))



dist_matrix_dict = {1:dist_matrix1, 2:dist_matrix2, 3:dist_matrix3, 4:dist_matrix4}
#dist_matrix_dict = {1:dist_matrix1, 2:dist_matrix2}



In [8]:
#read start point and end point to each road
#create a dictionary

df = pd.read_excel("shepp_coor_final_adjusted.xlsx", sheet_name="Sheet1")

In [9]:
road_coor = defaultdict()
road_dist = defaultdict()
for i, row in df.iterrows():
    start_coor = "({},{})".format(row["start_long"],row["start_lat"])
    end_coor = "({},{})".format(row["end_long"],row["end_lat"])
    road_coor[row["Inspection ID"]] = [start_coor, end_coor]
    road_dist[row["Inspection ID"]] = round(row['inspection km'],2)

In [10]:
shepp_cluster = defaultdict(list)
road_cat = defaultdict(list)
for i, row in df.iterrows():
    if isinstance(row['cluster'],str):
        cluster = [int(s) for s in row['cluster'].split() if s.isdigit()]
        for i in cluster:
            shepp_cluster[i].append(row['Inspection ID'])
    else:
        shepp_cluster[row['cluster']].append(row['Inspection ID'])
    road_cat[row['class']].append(row['Inspection ID'])

In [11]:
dist_matrix_dict[1].loc[road_coor['SHE_28'][0],road_coor['SHE_28'][1]]+\
dist_matrix_dict[1].loc['(145.41880628382472,-36.37819524436343)', road_coor['SHE_28'][0]]



45.681

In [13]:
def route_driving_distance(route, cluster):
    '''calculate total driving distance of route'''
    total_dist = 0
    pre_point = '(145.41880628382472,-36.37819524436343)'
    end_point = '(145.41880628382472,-36.37819524436343)'
    locs=[pre_point]
    for k in route:
        start = road_coor[k][0]
        end = road_coor[k][1]
        forward = dist_matrix_dict[cluster].loc[pre_point, start] +dist_matrix_dict[cluster].loc[start, end]
        backward = dist_matrix_dict[cluster].loc[pre_point, end] +dist_matrix_dict[cluster].loc[end, start]
        if forward < backward:
            pre_point = end
            total_dist += forward
            locs.append(start)
            locs.append(end)

        else:
            pre_point = start
            total_dist += backward
            locs.append(end)
            locs.append(start)
    total_dist = total_dist + dist_matrix_dict[cluster].loc[pre_point, end_point]
    locs.append(end_point)
    
    return total_dist,locs

In [14]:
def comb_inspection_distance(comb):
    '''calculate inspection distance of a road combination'''
    inspection_dist = 0
    for k in comb:
        inspection_dist += road_dist[k]
    return inspection_dist

In [15]:
from itertools import combinations, permutations

def shortest_distance(cluster: int):
    '''calculate the shortest driving distance for each possible scenario of road inspection within 
    a particular cluster'''
    shortest_distance = defaultdict(list)
    for i in range(1,len(shepp_cluster[cluster])+1):
        road_scenario = combinations(shepp_cluster[cluster],i)
        for j in road_scenario:
            route_perm = permutations(j)
            total_dist_dict = defaultdict(float)
            coordinates=defaultdict(list)
            inspect_dist = comb_inspection_distance(j)
            if inspect_dist <= 140 and inspect_dist >= 80:
                for route in list(route_perm):
                    dist,locations = route_driving_distance(route, cluster)
                    total_dist_dict[route] = dist
                    coordinates[route] = locations
                if min(total_dist_dict.values())<=300:
                    key_list = list(total_dist_dict.keys())
                    val_list = list(total_dist_dict.values())
                    position = val_list.index(min(total_dist_dict.values()))
                    shortest_route = key_list[position]
                    shortest_distance[str(j)] = [min(total_dist_dict.values()),inspect_dist, shortest_route, coordinates[shortest_route]]
    
    return shortest_distance

In [16]:
dist_c1 = shortest_distance(1)

KeyboardInterrupt: 

In [20]:
#clear empty dicts
def clear_empty_value(dict):
    for k in list(dict.keys()):
        if not dict[k]:
            del dict[k]
    return dict

In [22]:
dist_c2 = shortest_distance(2)

In [24]:
dist_c3 = shortest_distance(3)

In [26]:
dist_c4 = shortest_distance(4)

In [28]:
#define a dictionary that store which road combination beglongs to which cluster
comb_cluster_dict = {1:dist_c1.keys(), 2:dist_c2.keys(), 3:dist_c3.keys(), 4:dist_c4.keys()}

In [30]:
#get coords for ('SHE_01', 'SHE_07', 'SHE_26', 'SHE_04')
road_coor['SHE_01']+road_coor['SHE_07']+road_coor['SHE_26']+road_coor['SHE_04']

['(145.376957,-36.518167)',
 '(145.399107,-36.381874)',
 '(145.388727,-36.440749)',
 '(145.431021,-36.303724)',
 '(145.461966,-36.36458)',
 '(145.68616,-36.327422)',
 '(145.399244,-36.381019)',
 '(145.708043,-36.417435)']

In [31]:
dist_c1[('SHE_01', 'SHE_04', 'SHE_07', 'SHE_26')]

[]

In [32]:
dist_c1 = clear_empty_value(dist_c1)
dist_c2 = clear_empty_value(dist_c2)
dist_c3 = clear_empty_value(dist_c3)
dist_c4 = clear_empty_value(dist_c4)

In [33]:
pre_point='(145.41880628382472,-36.37819524436343)'

In [34]:
comb_distance = defaultdict()
comb_inspect = defaultdict()
comb_locations=defaultdict()

for dic in [dist_c1, dist_c2, dist_c3, dist_c4]:
    for key in dic:
        key = str(key)
        comb_distance[key] = dic[key][0]
        comb_inspect[key] = dic[key][1]
        loc=[]
        for a in (dic[key][3]):
            if(type(a)==str):
                latitude=round(float(a.split(",")[1][:-1]),5)
                longitude=round(float(a.split(",")[0][1:]),5)
            else:
                latitude=round(a[0],5)
                longitude=round(a[1],5)
            loc.append((latitude,longitude))
        comb_locations[key]=[dic[key][2],loc]
        
        

#### Model Formulation

In [36]:
from pulp import *
from gurobipy import *

In [37]:
#check available solvers
print(listSolvers(onlyAvailable=True))

Set parameter Username
Academic license - for non-commercial use only - expires 2023-10-12
No parameters matching '_test' found
['GUROBI', 'GUROBI_CMD', 'PULP_CBC_CMD']


In [38]:
road_comb = list(comb_distance.keys())
road_comb = [str(i) for i in road_comb]
road_id = list(df['Inspection ID'])
nweek = [1, 2, 3, 4]
ndays = [1, 2, 3, 4, 5]
clusters = [1, 2, 3, 4]

In [39]:
inspection_result=defaultdict(list)
total_driving_distance_result=defaultdict(list)
total_driving_distance_total_result=defaultdict(float)

In [40]:
dfs=[]
total_driving_distance_weight=[0,0.5,1,1.5,2,2.5,3,3.5,4]
for P in range(len(total_driving_distance_weight)):
    #create model
    m = Model('Inspection Scheduling')

    #initialise decision variables
    r = m.addVars(road_comb, nweek, ndays, vtype = GRB.BINARY, name = [[[f'r_{i}_{j}_{k}' for k in ndays] for j in nweek] for i in road_comb])

    c = m.addVars(clusters, nweek, ndays, vtype = GRB.BINARY, name = [[[f'c_{i}_{j}_{k}' for k in ndays] for j in nweek] for i in clusters] )

    z = m.addVars(road_id, vtype = GRB.BINARY, name = [f'z_{i}' for i in road_id])

    t = m.addVars(nweek, ndays[0:4], vtype = GRB.CONTINUOUS, name = [[f't_{j}_{k}' for k in range(1,5)] for j in nweek])

    wk= m.addVars(nweek)

    comb_dist_dict = defaultdict()
    comb_inspect_dict = defaultdict()
    for key in r:
        comb_dist_dict[key] = comb_distance[key[0]]
        comb_inspect_dict[key] = comb_inspect[key[0]]

    #add constraints

    #all road scenarios within a clutser will not occur if that cluster is not visited on a particular day
    for i in comb_cluster_dict:
        for j in nweek:
            for k in ndays:
                comb_sum = 0
                for comb in list(comb_cluster_dict[i]):
                    comb_sum += r.sum(comb, j, k)
                m.addConstr(comb_sum <= c.sum(i, j, k))

    #only one cluster is visted per day
    for j in nweek:
        for k in ndays:
            m.addConstr((c.sum('*', j, k) == 1), name= "one cluster per day")

    #RMC2
    road_rmc2 = road_cat[2]
    for road in road_rmc2:
        for j in nweek:
            sum1 = sum2 = sum3 = 0
            for comb in road_comb:
                if road in comb:
                    sum1 += r.sum(comb, j, 1) + r.sum(comb, j, 4)
                    sum2 += r.sum(comb, j, 2) + r.sum(comb, j, 5)
                    sum3 += r.sum(comb, j, 3)
            m.addConstr((sum1 == 2*z.sum(road)), name = "Monday and Thursday Inspection")
            m.addConstr((sum2 == 2 - 2*z.sum(road)), name = "Tuesday and Friday Inspection")
            m.addConstr((sum3 == 0), name = "no Wednesday Inspection")

    #RMC3
    road_rmc3 = road_cat[3]
    for road in road_rmc3:
        for k in ndays:
            week1 = week2 = week3 = week4 =0
            for comb in road_comb:
                if road in comb:
                    week1 += r.sum(comb, 1, k)
                    week2 += r.sum(comb, 2, k)
                    week3 += r.sum(comb, 3, k)
                    week4 += r.sum(comb, 4, k)
            m.addConstr(week1 == week2)
            m.addConstr(week2 == week3)
            m.addConstr(week3 == week4)
        for j in nweek:
            total = 0
            for comb in road_comb:
                if road in comb:
                    total += r.sum(comb, j, '*')
            m.addConstr((total == 1), name = "once a week")

    #RMC4
    road_rmc4 = road_cat[4]
    for road in road_rmc4:
        total1 = 0
        for k in ndays:
            w1 = w2 = w3 = w4 = 0
            for comb in road_comb:
                if road in comb:
                    w1 += r.sum(comb, 1, k)
                    w2 += r.sum(comb, 2, k)
                    w3 += r.sum(comb, 3, k)
                    w4 += r.sum(comb, 4, k)
            m.addConstr(w1 == w3)
            m.addConstr(w2 == w4)
        for comb in road_comb:
            if road in comb:
                total1 += r.sum(comb, '*','*')
        m.addConstr(total1 == 2, name = "once very two weeks")

    #RMC5
    road_rmc5 = road_cat[5]
    total2 = 0
    for road in road_rmc5:
        for comb in road_comb:
            if road in comb:
                total2 += r.sum(comb, '*','*')
        m.addConstr(total2 == 1, name = "once very four weeks")

    #each road cannot be inspected more than once on a day
    for road in road_id:
        for j in nweek:
            for k in ndays:
                sum4 = 0
                for comb in road_comb:
                    if road in comb:
                        sum4 += r.sum(comb,j,k)
                m.addConstr((sum4 <= 1), name = "each road cannot be inspect more than once")

    #absolute difference of inspection km between days
    for j in nweek:
        for k in ndays[0:4]:
            m.addConstr(t.sum(j,k) >= r.prod(comb_inspect_dict,'*',j,k+1)-r.prod(comb_inspect_dict,'*',j,k))
            m.addConstr(t.sum(j,k) >= r.prod(comb_inspect_dict,'*',j,k)-r.prod(comb_inspect_dict,'*',j,k+1))

    #absolute difference of inspection km between weeks
    m.addConstrs(wk.sum(j)>=(r.prod(comb_inspect_dict,'*',j+1,"*")-r.prod(comb_inspect_dict,'*',j,"*")) for j in nweek[0:3])
    m.addConstrs(wk.sum(j)>=(r.prod(comb_inspect_dict,'*',j,"*")-r.prod(comb_inspect_dict,'*',j+1,"*")) for j in nweek[0:3])

    #objective function
    obj_func = t.sum() + (wk.sum()) + total_driving_distance_weight[P]* r.prod(comb_dist_dict, '*', '*', '*')
   
    #############################################################################################################
    m.setObjective(obj_func, GRB.MINIMIZE)
    m.Params.MIPGap = 0.05
    m.Params.TimeLimit = 200
    m.optimize()

#############################################################################################################
    import re
    print('obj=', m.ObjVal)
    comb = []
    for v in m.getVars():
        if v.x>0 and v.varName.startswith('r'):
            print(v.varName, ':', v.x)
            res  = re.findall(r'\(.*?\)', v.varName)
            comb.append(res[0])
    #############################################################################################################
    final_dist = defaultdict()
    for i in comb:
        final_dist[i] = comb_distance[i]
    total_distance=0
    for key in comb:
        total_distance += comb_distance[key]

    from collections import defaultdict
    road_schedules = defaultdict(list)
    #############################################################################################################
    for v in m.getVars():
        if v.x>0 and v.varName.startswith('r'):
            #anything inside '()'
            roads  = re.findall(r'\(.*?\)', v.varName)
            road_list=roads[0][1:-1].split(",")
            week=(v.VarName[-4:][1])
            day=(v.VarName[-4:][3])
            for road in road_list:
            #Find road name inside '' 
                road_name=re.findall(r"'(.*?)'", road)[0]
                road_schedules[road_name].append((week,day))
#############################################################################################################
    cols=['  ']*2+['W1']+['  ']*2+['  ']*2+['W2']+['  ']*2+['  ']*2+['W3']+['  ']*2+['  ']*2+['W4']+['  ']*2
    df=pd.DataFrame(columns=cols,index=[['Roads/ Days']+[i for i in road_dist.keys()]])
    #set all values of df to 0
    df.iloc[0,:]=['[ day1','day2','day3','day4','day5  ]']*4
    df=df.fillna('')
    def get_index_num(df,road):
        if(road[-2]!='0'):
            row_index=int(road[-2:])
        else:
            row_index=int(road[-1])
        return row_index

    for road in road_schedules:
        for week,day in road_schedules[road]:
            col_index=(int(week)-1)*5+int(day)-1
            row_number=get_index_num(df,road)
            df.iloc[row_number,col_index]=road_dist[road]

#############################################################################################################
    df.loc['Daily_inspection'] = ''
    df.loc['Daily_driving_distance']=''
    df.loc['Daily visiting Locations (Shortest Distance route)'] =[list() for x in range(len(df.columns))]
    df.loc['Location Coordinates']=[list() for x in range(len(df.columns))]
    df
    # #loop through each column and sum the values
    for i in range(0,df.shape[1]):
    #     #loop through each row
        total=0
        for j in range(1,31):
            if(df.iloc[j,i]!=''):
                total+=df.iloc[j,i]
        df.iloc[df.index.get_loc('Daily_inspection'),i]=float(total)
        inspection_result[P].append(total)
    df=df.replace(0,'')
    # #change the row of total to 0 if it's blank
    for j in range(0,df.shape[1]):
        if(df.iloc[df.index.get_loc('Daily_inspection'),j]==''):
            df.iloc[df.index.get_loc('Daily_inspection'),j]=0
    pd.set_option('display.max_columns', None)
    # #sum up the all columns in df and put it in the last row of df
    # df.iloc[df.index.get_loc('Daily_inspection'),10]=df.iloc[-3].sum()


    for j in range(0,df.shape[1]):
        value_list=[]
        for i in range(1,31):
            if(df.iloc[i,j]!=''):
                if(i<=9):
                    value_list.append('0'+str(i))
                else:
                    value_list.append(str(i))
        route_list="\"("
        for each in value_list:
            if(each!=value_list[-1]):
                route_list+=('\'SHE_'+each+'\', ')
            else:
                route_list+=('\'SHE_'+each+'\')\"')
        #ignore the escaping character for route_list
        if(route_list!="\"("):
            route_list=eval(route_list)
            route_list=str((route_list))
        else:
            route_list=""
        if(route_list in comb_locations):
            Locations=comb_locations[route_list]
            df.iloc[df.index.get_loc('Daily visiting Locations (Shortest Distance route)'),j].append(Locations[0])
            df.iloc[df.index.get_loc('Location Coordinates'),j].append(Locations[1])

        else:
            Locations=["NA","NA"]
            df.iloc[df.index.get_loc('Daily visiting Locations (Shortest Distance route)'),j].append(Locations[0])
            df.iloc[df.index.get_loc('Location Coordinates'),j].append(Locations[1])
        if(route_list in final_dist):
            df.iloc[df.index.get_loc('Daily_driving_distance'),j]=final_dist[route_list]
        else:
            df.iloc[df.index.get_loc('Daily_driving_distance'),j]="NA"
        
    total_driving_distance_total_result[P]=total_distance
    dfs.append(df)



Set parameter MIPGap to value 0.05
Set parameter TimeLimit to value 200
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 1139 rows, 33491 columns and 793896 nonzeros
Model fingerprint: 0x4ce3b96c
Variable types: 20 continuous, 33471 integer (33471 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Presolve removed 642 rows and 3348 columns
Presolve time: 1.00s
Presolved: 497 rows, 30143 columns, 531950 nonzeros
Variable types: 0 continuous, 30143 integer (30094 binary)

Root relaxation: objective 0.000000e+00, 2783 iterations, 1.19 seconds (3.15 work units)
Total elapsed time = 5.06s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0

In [41]:
dfs[2]

,,,W1,,,,,W2,,,,,W3,,,,,W4,,
"(Roads/ Days,)",[ day1,day2,day3,day4,day5 ],[ day1,day2,day3,day4,day5 ],[ day1,day2,day3,day4,day5 ],[ day1,day2,day3,day4,day5 ]
"(SHE_01,)",,15.5,,,15.5,,15.5,,,15.5,,15.5,,,15.5,,15.5,,,15.5
"(SHE_02,)",80.3,,,80.3,,80.3,,,80.3,,80.3,,,80.3,,80.3,,,80.3,
"(SHE_03,)",,34.3,,,,,34.3,,,,,34.3,,,,,34.3,,,
"(SHE_04,)",,,,,29.1,,,,,29.1,,,,,29.1,,,,,29.1
"(SHE_05,)",,,,2.76,,,,,2.76,,,,,2.76,,,,,2.76,
"(SHE_06,)",,,,,5.46,,,,,5.46,,,,,5.46,,,,,5.46
"(SHE_07,)",,,18.93,,,,,18.93,,,,,18.93,,,,,18.93,,
"(SHE_08,)",1.14,,,,,1.14,,,,,1.14,,,,,1.14,,,,
"(SHE_09,)",,,51.95,,,,,,,,,,51.95,,,,,,,


In [1]:
for i in range(len(total_driving_distance_weight)):
    dfs[i].to_excel('/Users/kuan/Desktop/Downer Project Final/Schedule Results/N_result('+str(i+1)+').xlsx')

NameError: name 'total_driving_distance_weight' is not defined